# Proyecto 1
* Flavio Galán - 22386
* Josue Say - 22801
* Isabella Miralles

## Descarga de los Datos

Para descargar los datos se utiliza Selenium y Python, además se tiene un sistema de cacheo que funciona en base al archivo `links_cache.txt`. Si este archivo ya existe entonces no se realizará ninguna descarga de los datos. Los datos se guardan en varios archivos txt. Cada uno representa la columna del dataframe y todos los datos de esa columna. No se tienen en un formato JSON ni CSV sino que se guardan directamente en formato python para la facilidad de extracción usando el mismo lenguaje. Ya que estos son los datos sucios no hay ningún problema con guardarlos así, los datos limpios ya se guardarán en un formaton más estandarizado.

### Importaciones y configuración para el web scraping


In [ ]:
import os
import time
# import subprocess (se reemplazo con el uso de os para cualquier sistema operativo)
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

### Configuración de rutas y archivo de caché

In [ ]:
cache_file_name = "./cache/links_cache.txt"
df_cache_file = "./cache/dataframe_cache.txt"
zip_dir = "zips/"
files_dir = "data/"
csv_file_path = "./data/data_unified.csv"
reports = "reportes/"


os.makedirs("cache", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("zips", exist_ok=True)
os.makedirs("reportes", exist_ok=True)

### Inicialización de estructuras para almacenar los datos

In [ ]:
codigo = []
distrito = []
departamento = []
municipio = []
establecimiento = []
direccion = []
telefono = []
supervisor = []
director = []
nivel = []
sector = []
area = []
status = []
modalidad = []
jornada = []
plan = []
departamental = []
agggrArrays = [
    codigo,
    distrito,
    departamento,
    municipio,
    establecimiento,
    direccion,
    telefono,
    supervisor,
    director,
    nivel,
    sector,
    area,
    status,
    modalidad,
    jornada,
    plan,
    departamental,
]
filenames = [
    "codigo",
    "distrito",
    "departamento",
    "municipio",
    "establecimiento",
    "direccion",
    "telefono",
    "supervisor",
    "director",
    "nivel",
    "sector",
    "area",
    "status",
    "modalidad",
    "jornada",
    "plan",
    "departamental",
]

### Carga de datos (web scrapping)

In [ ]:
if not os.path.exists(cache_file_name):
    with webdriver.Firefox() as driver:
        driver.implicitly_wait(3)

        driver.get("http://www.mineduc.gob.gt/BUSCAESTABLECIMIENTO_GE/")
        assert "Búsqueda de centros" in driver.title

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']")
            )
        )
        deptSelect = driver.find_element(
            By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']"
        )
        deptSelect = Select(deptSelect)
        depts = deptSelect.options
        print("Getting depts")
        for idx in range(len(depts) - 1):
            print("Selecting dept", idx + 1)
            deptSelect = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_cmbDepartamento']"
            )
            deptSelect = Select(deptSelect)
            deptSelect.select_by_index(idx + 1)
            print("Finding education level")
            levelSelect = driver.find_element(
                By.XPATH, '//*[@id="_ctl0_ContentPlaceHolder1_cmbNivel"]'
            )
            levelSelect = Select(levelSelect)
            print("Selecting diversificado")
            levelSelect.select_by_value("46")  # 46 es Diversificado
            print("Finding search button")
            btn = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_IbtnConsultar']"
            )
            btn.click()
            print("Clicking button")

            # Wait for results
            print("Esperando por resultados")
            time.sleep(5)
            print("Asumimos que se obtuvieron los resultados")

            table = driver.find_element(
                By.XPATH, "//*[@id='_ctl0_ContentPlaceHolder1_dgResultado']"
            )
            rows = table.find_elements(By.XPATH, ".//tr")
            for rowIdx in range(
                1, len(rows) - 1
            ):  # La última fila siempre es una vacía
                cells = rows[rowIdx].find_elements(By.XPATH, ".//td")

                for cellIdx in range(1, len(cells)):
                    agggrArrays[cellIdx - 1].append(
                        cells[cellIdx].get_attribute("textContent")
                    )

            # print("Obtained following names")
            # print(establecimiento)
            # exit(1)

        print("Saving cache...")
        os.makedirs(zip_dir, exist_ok=True)
        for idx in range(len(filenames)):
            filename = filenames[idx]
            data = agggrArrays[idx]

            lines = ["["]
            for val in data:
                line = f"\t'''{val}''',\n"
                lines.append(line)
            lines.append("]")

            with open(zip_dir + filename + ".txt", "w") as file:
                file.writelines(lines)

        os.makedirs(os.path.dirname(cache_file_name), exist_ok=True)
        with open(cache_file_name, "w") as file:
            file.write("DELETE ME IF YOU WANT TO REDOWNLOAD DATA!")


### Carga de datos al DataFrame

Este bloque verifica si existen los archivos de caché (`links_cache.txt` y `dataframe_cache.txt`). Si no hay caché, construye el `DataFrame` desde los datos extraídos y guarda un `.csv`. Si los cachés ya existen, carga directamente el `DataFrame` desde el archivo CSV, evitando repetir procesos.

In [ ]:
def loadDfCache(
    filenames=None,
    agggrArrays=None,
    csvFile=csv_file_path,
    dfCache=df_cache_file
):
    if os.path.exists(csvFile):
        df = pd.read_csv(csvFile, encoding="utf-8-sig")
        print("DataFrame loaded from CSV:")
        print(df)

        if not os.path.exists(dfCache):
            with open(dfCache, "w") as f:
                f.write("DataFrame cache created.")
    else:
        df = pd.DataFrame(
            {colName: colData for (colName, colData) in zip(filenames, agggrArrays)}
        )
        print("The resulting DataFrame is:")
        print(df)

        df.to_csv(csvFile, index=False, encoding="utf-8-sig")

        with open(dfCache, "w") as f:
            f.write("DataFrame cache created.")

    return df

In [ ]:
df = loadDfCache(filenames, agggrArrays)

## Estructura del Conjunto de Datos Crudo

En esta etapa se realiza un análisis exploratorio preliminar del conjunto de datos descargado, con el objetivo de conocer su estructura general. Se identifica la cantidad de filas y columnas, la presencia de datos duplicados, valores nulos por variable, y los tipos de datos registrados.

In [ ]:
def generateDataReport(df: pd.DataFrame, saveToFile: bool = True):
    """
    Genera un resumen general del DataFrame con información básica.
    
    Parámetros:
    - df: DataFrame de entrada.
    - saveToFile: Si es True, guarda el reporte en 'reportes/reporte_general.txt'.

    Retorna:
    - Lista de líneas del reporte como strings.
    """
    if saveToFile:
        os.makedirs(reports, exist_ok=True)

    num_filas, num_columnas = df.shape
    duplicados_filas = df.duplicated().sum()
    nulos_por_columna = df.isnull().sum()
    tipos_datos = df.dtypes

    # Armar el contenido del reporte
    reporte = []
    reporte.append(f"Total de filas: {num_filas}")
    reporte.append(f"Total de columnas: {num_columnas}")
    reporte.append(f"Filas duplicadas: {duplicados_filas}")
    reporte.append("\nValores nulos por columna:")
    reporte.extend([f"{col}: {nulos}" for col, nulos in nulos_por_columna.items()])
    reporte.append("\nTipos de datos por columna:")
    reporte.extend([f"{col}: {tipo}" for col, tipo in tipos_datos.items()])

    if saveToFile:
        with open(os.path.join(reports, "reporte_general.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(reporte))
        print("Reporte generado en 'reportes/reporte_general.txt'")

    return reporte

In [ ]:
is_generate_report = True
report_lines = generateDataReport(df, saveToFile=is_generate_report)
# print("\n".join(report_lines))

## Análisis Exploratorio Inicial

En esta etapa se realizará un análisis exploratorio más profundo del conjunto de datos crudo, con el objetivo de entender mejor su contenido, diversidad y consistencia.

## Descripción de los Datos
Se describen los datos y las transformaciones necesarias que se les realizaran a continuación.

## Limpieza de los datos
Se procede a ejecutar las transformaciones previamente ideadas y a unificar todos los datasets en uno solo.

### Sección 1

### Sección 2